In [1]:
import openai
import pandas as pd
import numpy as np
import string

In [22]:
model_embed = "nomic-ai/nomic-embed-text-v1.5-GGUF/nomic-embed-text-v1.5.Q5_K_M.gguf"
model_embed2 = "second-state/Nomic-embed-text-v1.5-Embedding-GGUF/nomic-embed-text-v1.5-f16.gguf"

In [34]:

def embed_text(text):
    #client = openai.Client() 
    client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
    response = client.embeddings.create(
        model=model_embed,
        input=text,
        encoding_format="float"
    )
    return response.data[0].embedding


In [35]:
df_corpus_embed_llm = pd.read_csv('data/corpus_clean_pre_embed.csv',sep="|")

In [36]:
df_corpus_embed_llm['embeddings_LLM'] = df_corpus_embed_llm['texto'].apply(embed_text)

In [37]:
# Defining cosine similarity 
def compute_cosine_similarity(a,b): 
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return (cos_sim)

In [41]:
uq = 'que cuota anual tiene una tarjeta de debito'
e = embed_text(uq)
print (e)
df_corpus_embed_llm['score_LLM'] = df_corpus_embed_llm['embeddings_LLM'].apply(lambda x: compute_cosine_similarity(x, e))

[0.045150142163038254, 0.05633664131164551, -0.15408384799957275, -0.025570666417479515, 0.010858913883566856, -0.0095968684181571, -0.05970063805580139, -0.03518896549940109, -0.07629840821027756, -0.014896033331751823, -0.047502052038908005, -0.015588396228849888, 0.07524123787879944, -0.03417830541729927, 0.05093612149357796, -0.015838880091905594, 0.017663855105638504, -0.05776048079133034, -0.02558879554271698, 0.06941573321819305, 0.03108353167772293, -0.045000337064266205, -0.07244519144296646, -0.022003790363669395, 0.0561455637216568, -0.041678521782159805, -0.006296069361269474, 0.09451863169670105, 0.02648899517953396, -0.042348314076662064, -0.042539648711681366, -0.07271017879247665, -0.008285313844680786, 0.03393807262182236, 0.04575464874505997, -0.036075398325920105, 0.04029622673988342, 0.057254090905189514, 0.01829293929040432, -0.0017755271401256323, 0.0025365063920617104, -0.028610989451408386, -0.03627782687544823, -0.026384955272078514, 0.022310268133878708, 0.010

In [42]:
df_corpus_embed_llm[['CODIGO_FAQ','embeddings_LLM','score_LLM','motivo']].to_csv('data/variaciones_embedding_llm_uoc_20240427.csv',sep="|", index=None)

In [43]:
df_corpus_embed_llm[['CODIGO_FAQ','embeddings_LLM','score_LLM','motivo']].to_pickle('data/variaciones_embedding_llm_uoc_20240427.pkl')

In [44]:
df_corpus_embed_llm.head()

,CODIGO_FAQ,variaciones,motivo,texto,embeddings_LLM,score_LLM
0,433243,"['Tarjeta para empleados Lidl', 'solicitar UNA...",NaN,tarjeta para empleados lidl solicitar una tarj...,"[-0.0014396568294614553, 0.07590703666210175, ...",0.731236
1,419915,"['Anular cobro liquidación tarjeta oficina', '...",NaN,anular cobro liquidacion tarjeta oficina proce...,"[0.022069312632083893, 0.029363106936216354, -...",0.662355
2,421998,"['motivo cambio de límite Tarjeta', 'como renu...",NaN,motivo cambio de limite tarjeta como renunciar...,"[0.08342310041189194, 0.08172677457332611, -0....",0.599550
3,423029,['Qué hago para cambiar la forma de cobro de l...,NaN,que hago para cambiar la forma de cobro de la ...,"[-0.009927797131240368, 0.05764125660061836, -...",0.589149
4,422618,['Por dónde veo los plazos de recepción de las...,NaN,por donde veo los plazos de recepcion de las t...,"[0.027231913059949875, 0.06630170345306396, -0...",0.666546


In [45]:
print("score_LLM: \n",df_corpus_embed_llm['score_LLM'].sort_values(ascending=False))

score_LLM: 
 59     0.798521
104    0.794803
38     0.781082
164    0.765781
133    0.757367
         ...   
283    0.518939
135    0.513585
382    0.512450
67     0.502481
424    0.492921
Name: score_LLM, Length: 439, dtype: float64


In [46]:
# Get index of the 5 highest score_mini's rows 
indices_max_scores_llm = df_corpus_embed_llm['score_LLM'].nlargest(5).index

# print scores and codes
codes_max_scores_llm = df_corpus_embed_llm.loc[indices_max_scores_llm, ['CODIGO_FAQ','score_LLM']] 
print(codes_max_scores_llm)

     CODIGO_FAQ  score_LLM
59       435574   0.798521
104      418183   0.794803
38       442151   0.781082
164      446600   0.765781
133      422278   0.757367


In [ ]:
df_corpus_llm_final = df_corpus_embed_llm.rename(columns={'embeddings_LLM':'embeddings'})
df_corpus_llm_final.to_csv('../FastAPI/db/data/embedding/variaciones_embedding_llm_20240427.csv',sep="|", index=None)
df_corpus_llm_final.to_pickle('../FastAPI/db/data/embedding/variaciones_embedding_llm_20240427.pkl')